In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import os
import seaborn as sns
from keras_preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import Sequence
from sklearn.model_selection import train_test_split
import cv2
from os import listdir
import skimage.exposure

In [2]:
train=pd.read_csv("Dataset/Training_set.csv")

In [3]:
lb=LabelEncoder()
train["food_no"]=lb.fit_transform(train["label"])

In [4]:
image=cv2.imread("Dataset/train/Image_1.jpg")
image.shape

(224, 224, 3)

In [5]:
X_train, X_valid = train_test_split(train, test_size=0.1,stratify=train["label"],shuffle=True)

In [6]:
train_datagen=ImageDataGenerator(rescale=1./255,)

In [7]:
train_generator=train_datagen.flow_from_dataframe(
                                                        dataframe=X_train,
                                                        directory="Dataset/train",
                                                        x_col="filename",
                                                        y_col="label",
                                                        batch_size=16,
                                                        seed=42,
                                                        shuffle=True,
                                                        class_mode="categorical",
                                                        target_size=(224,224))

Found 5151 validated image filenames belonging to 8 classes.


In [8]:
valid_datagen=ImageDataGenerator(rescale=1./255.,)

valid_generator=valid_datagen.flow_from_dataframe(
                                                    dataframe=X_valid,
                                                    directory="Dataset/train",
                                                    x_col="filename",
                                                    y_col="label",
                                                    batch_size=16,
                                                    seed=42,
                                                    shuffle=True,
                                                    class_mode="categorical",
                                                    target_size=(224,224))

Found 573 validated image filenames belonging to 8 classes.


In [14]:
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input

import tensorflow as tf

In [15]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]
# add preprocessing layer to the front of VGG
vgg_16 = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False,pooling='avg')

In [16]:
# don't train existing weights
for layer in vgg_16.layers:
    layer.trainable = False

In [17]:
# our layers - you can add more if you want
x = tf.keras.layers.Flatten()(vgg_16.output)
prediction = tf.keras.layers.Dense(8, activation='softmax')(x)
model = tf.keras.models.Model(inputs=vgg_16.input, outputs=prediction)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [18]:
# tell the model what cost and optimization method to use
model.compile(
loss='categorical_crossentropy',
optimizer='adam',
metrics=['accuracy']
)

In [29]:
model.fit(
    train_generator,
    epochs=2,
    validation_data=valid_generator
    )

Epoch 1/2
322/322 [==============================] - 2121s 7s/step - loss: 1.6749 - accuracy: 0.3491 - val_loss: 1.6515 - val_accuracy: 0.3805
Epoch 2/2
322/322 [==============================] - 2186s 7s/step - loss: 1.6333 - accuracy: 0.3586 - val_loss: 1.6185 - val_accuracy: 0.3857


In [30]:
test=pd.read_csv("Dataset/Testing_set.csv")
sub=pd.read_csv("Dataset/sample_submission.csv")

In [31]:
test_datagen=ImageDataGenerator(rescale=1./255.,)
test_generator=valid_datagen.flow_from_dataframe(
                                                        dataframe=test,
                                                        directory="Dataset/test",
                                                        x_col="filename",
                                                        y_col=None,
                                                        batch_size=1,
                                                        seed=42,
                                                        shuffle=False,
                                                        class_mode=None,
                                                        target_size=(224,224))

Found 1908 validated image filenames.


In [32]:
pred1=np.argmax(model.predict(test_generator,verbose=1),axis=1)

1908/1908 [==============================] - 992s 520ms/step


In [33]:
prediction=lb.inverse_transform(pred1)
sub["label"]=prediction

In [34]:
sub.label.value_counts()

Density1Malignant    1176
Density2Malignant     701
Density3Benign         31
Name: label, dtype: int64

In [27]:
sub.to_csv("pred-vgg161.csv",index=False)